In [17]:
db = '/media/ttsmith/DB/Dropbox/'
tmp_dir = db + 'TMP/'

import pandas as pd
import xarray as xr
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

# zarr_gdf = gpd.read_file(db + 'Glacier Velocity/ITS_LIVE/catalog_v02.json')
zarr_gdf = "https://its-live-data.s3.amazonaws.com/datacubes/catalog_v02.json"

In [18]:
def choose_loc(zarr_gdf, xy):
    pt_loc = Point(xy)
    xy_proj = gpd.GeoDataFrame(geometry=[pt_loc],crs='epsg:4326')
    print(xy_proj)
    subset = gpd.sjoin(xy_proj, zarr_gdf, how="inner", predicate='within')
    return subset.zarr_url.values[0]

def pull_xy(ds, xy): #SINGLE LOCATION
    xy = Point(xy)
    xy_proj = gpd.GeoDataFrame(geometry=[xy],crs='epsg:4326').to_crs(ds.projection)
    data = ds.sel(x=xy_proj.geometry.x.values[0],y=xy_proj.geometry.y.values[0],method='nearest')
    velocity = data.v.values
    timing = np.array([pd.Timestamp(x) for x in data.date_center.values])
    error = data.v_error.values
    dt = data.date_dt.values

    d0 = np.array([pd.Timestamp(x) for x in data.acquisition_date_img1.values])
    d1 = np.array([pd.Timestamp(x) for x in data.acquisition_date_img2.values])

    days = np.array([convert_to_days(x) for x in dt])

    #Remove locked (less than 1) pixels
    velocity[velocity <= 0.5] = np.nan

    #Also filter for local large outliers (negative, missed points)
    velocity[velocity < np.nanpercentile(velocity, 2)] = np.nan

    idx = ~np.isnan(velocity)
    velocity, timing, error, days = velocity[idx], timing[idx], error[idx], days[idx]
    d0, d1 = d0[idx], d1[idx]

    #Filter to Landsat-8 Era
    idx = np.where(timing > pd.Timestamp(2013,4,10))
    velocity, timing, error, days = velocity[idx], timing[idx], error[idx], days[idx]
    d0, d1 = d0[idx], d1[idx]

    return velocity, timing, error, days, d0, d1

def convert_to_days(x):
    x = np.timedelta64(x, 'ns')
    days = x.astype('timedelta64[D]')
    return days / np.timedelta64(1, 'D')

def create_consensus_series(midpoints, v_obs, v_uncert, time_spacing, n_samp):
    def create_uniform_dates(start_year, end_year, time_spacing=16):
        #Define uniform spacing for whole period
        uniform_times = []
        for year in range(start_year, end_year + 1):
            year_start = pd.Timestamp(str(year) + '-01-01')
            year_days = 366 if pd.Timestamp(str(year) + '-12-31').is_leap_year else 365
            steps = np.arange(0, year_days, time_spacing)  #X-day intervals
            uniform_times.extend((year_start + pd.to_timedelta(steps, unit="D")).tolist())

        uniform_times = np.array(uniform_times)

        #Convert uniform times into linear days distance
        reference_date = pd.Timestamp(str(start_year) + '-01-01')
        uniform_times_numeric = np.array([(t - reference_date).days for t in uniform_times])
        n_uniform = len(uniform_times_numeric) - 1  #Number of steps to invert onto

        return uniform_times, uniform_times_numeric, n_uniform, reference_date

    uniform_times, uniform_times_numeric, n_uniform, reference_date = create_uniform_dates(midpoints.index.year.min(), midpoints.index.year.max(), time_spacing)
    midpoints_numeric = (midpoints - reference_date).dt.days

    #Get which bin each midpoint maps to
    inversion_dates = np.array([np.argmin(np.abs(uniform_times_numeric - mid)) for mid in midpoints_numeric])
    unique_dates, ct = np.unique(inversion_dates, return_counts=True)

    #Choose a min number of points per bin
    sample_size = int(np.nanpercentile(ct, 5))

    def get_vel_series(inversion_dates, unique_dates, v_obs, v_uncert, n=100):
        def get_series(inversion_dates, unique_dates, v_obs, v_uncert):
            #For each bin, choose a random sample of velocities and their weights
            avgd = []
            for unique_id in unique_dates:
                idx = np.where(inversion_dates == unique_id)
                v, ve = v_obs[idx], v_uncert[idx]
                rsub = np.random.randint(0, high=v.shape[0], size=sample_size)
                vc, vec = v[rsub], ve[rsub]
                avg = np.average(vc, weights=vec)
                avgd.append(avg)
            return avgd

        emp = np.empty((unique_dates.shape[0], n))
        for i in range(n):
            avgd = get_series(inversion_dates, unique_dates, v_obs, v_uncert)
            emp[:,i] = avgd
        return emp

    averages = get_vel_series(inversion_dates, unique_dates, v_obs, v_uncert, n=n_samp)
    consensus = np.nanmean(averages, axis=1)
    f_ser = pd.Series(consensus, index=uniform_times[unique_dates])
    return f_ser, averages, uniform_times[unique_dates]

In [19]:
#MY VERSION
lon = -140.84746241412898
lat = 60.08936753843976
print(choose_loc(zarr_gdf, [lon,lat]))
ds = xr.open_dataset(choose_loc(zarr_gdf, [lon,lat]),engine="zarr")
velocity, timing, error, days, d0, d1 = pull_xy(ds, [lon,lat])
timing = [pd.Timestamp(x) for x in timing]
midpoints = pd.Series(timing,index=timing) #Midpoints

time_spacing = 4
f_ser, sample_series, series_index = create_consensus_series(midpoints, velocity, error, time_spacing=time_spacing, n_samp=100)
f_ser.plot()

time_spacing = 16
f_ser, sample_series, series_index = create_consensus_series(midpoints, velocity, error, time_spacing=time_spacing, n_samp=100)
f_ser.plot()

                      geometry
0  POINT (-140.84746 60.08937)


ValueError: 'right_df' should be GeoDataFrame, got <class 'str'>

In [ ]:
#Simple Version
from src.ticoi.core import ticoi_one_pixel
lon = -140.84746241412898
lat = 60.08936753843976
pick_date = ["2013-01-01", "2025-01-01"]# date range to study
coef = 100  # Regularization coefficient to be used
delete_outliers = {"median_angle": 45} #Remove the observation if its direction is 45° away from the direction of the median vector
save = True  # Save the results and figures
show = True  # Plot some figures
option_visual = ["obs_magnitude", "invertvv_overlaid", "quality_metrics"] #check README_visualization_pixel_output to see the different options .
result_quality = [
    "Error_propagation",
    "X_contribution",
]  # Criterium used to evaluate the quality of the results: ("Error_propagation": the initial error given in the dataset is propagated through the inversion; "X_contribution" correspond to the number of observed velocity used to estimate each estimated value

url_ls = choose_loc(zarr_gdf,[lon,lat])
data,dataf,dataf_lp = ticoi_one_pixel(cube_name=url_ls,i=lon,j=lat,path_save=tmp_dir,save=save,show=show,option_visual=option_visual,load_kwargs={"pick_date":pick_date,"buffer": [lon, lat, 0.1]},load_pixel_kwargs={"visual":show},preData_kwargs={"delete_outliers":delete_outliers},inversion_kwargs = {"coef":coef,"result_quality":result_quality,"visual":show},interpolation_kwargs = {"result_quality":result_quality})


In [ ]:
# Longer version, fails in data loading somewhere
lon = -140.84746241412898
lat = 60.08936753843976
url_ls = choose_loc(zarr_gdf,[lon,lat])

import time
from src.ticoi.cube_data_classxr import CubeDataClass

pick_date = ["2013-01-01", "2025-01-01"]# date range to study
coef = 100  # Regularization coefficient to be used
delete_outliers = {"median_angle": 45} #Remove the observation if its direction is 45° away from the direction of the median vector

apriori_weight = False  # Use the error as apriori
interval_output = 30  # temporal sampling of the output results
unit = 365  # 1 for m/d, 365 for m/y
regu = "1accelnotnull"  # Regularization method.s to be used (for each flag if flags is not None) : 1 minimize the acceleration, '1accelnotnull' minize the distance with an apriori on the acceleration computed over a spatio-temporal filtering of the cube
result_quality = [
    "Error_propagation",
    "X_contribution",
]  # Criterium used to evaluate the quality of the results ('X_count', 'Norm_residual', 'X_contribution', 'Error_propagation')
verbose = True  # Print information throughout TICOI processing

proj = "EPSG:4326"
load_kwargs = {
    "chunks": {},
    "conf": False,  # If True, confidence indicators will be put between 0 and 1, with 1 the lowest errors
    "subset": None,  # Subset of the data to be loaded ([xmin, xmax, ymin, ymax] or None)
    "buffer": None, # Area to be loaded around the pixel ([longitude, latitude, buffer size] or None)
    "nearest": [lon, lat], #Loc for nearest sample
    "pick_date": pick_date,  # Select dates ([min, max] or None to select all)
    "pick_sensor": None,  # Select sensors (None to select all)
    "pick_temp_bas": None,  # Select temporal baselines ([min, max] in days or None to select all)
    "proj": proj,  # EPSG system of the given coordinates
    "verbose": verbose,  # Print information throughout the loading process
}

start = time.time()
cube = CubeDataClass()
cube.load(url_ls, **load_kwargs)
print(f"[Data loading] Cube of dimension (nz,nx,ny) : ({cube.nz}, {cube.nx}, {cube.ny}) ")
stop = time.time()
print(f"[Data loading] Loading the Zarr took {round((stop - start), 4)} s")

proj = "EPSG:3413"  # EPSG system of the given coordinates
preData_kwargs = {
    "smooth_method": "savgol",  # Smoothing method to be used to smooth the data in time ('gaussian', 'median', 'emwa', 'savgol')
    "s_win": 3,  # Size of the spatial window
    "t_win": 90,  # Time window size for 'ewma' smoothing
    "sigma": 3,  # Standard deviation for 'gaussian' filter
    "order": 3,  # Order of the smoothing function
    "unit": 365,  # 365 if the unit is m/y, 1 if the unit is m/d
    "delete_outliers": delete_outliers,  # Delete the outliers from the data according to one (int or str) or several (dict) criteriums
    "flag": None,  # Divide the data in several areas where different methods should be used
    "dem_file": None,  # Path to the DEM file for calculating the slope and aspect
    "regu": regu,  # Regularization method.s to be used (for each flag if flags is not None) : 1 minimize the acceleration, '1accelnotnull' minize the distance with an apriori on the acceleration computed over a spatio-temporal filtering of the cube
    "solver": "LSMR_ini",  # Solver for the inversion
    "proj": proj,  # EPSG system of the given coordinates
    "velo_or_disp": "velo",  # Type of data contained in the data cube ('disp' for displacements, and 'velo' for velocities)
    "verbose": True,  # Print information throughout the filtering process
}

start = time.time()
# Filter the cube (compute rolling_mean for regu=1accelnotnull)
obs_filt, flag = cube.filter_cube_before_inversion(**preData_kwargs)
stop = time.time()
print(f"Preprocess Filtering took {round((stop - start), 4)} s")

show = True  # Plot some figures
save = True  # Save the results and figures
option_visual = ["obs_magnitude", "invertvv_overlaid", "quality_metrics"] #check README_visualization_pixel_output to see the different options .
path_save = save_dir + 'TICOI/'
load_pixel_kwargs = {
    "regu": regu,  # Regularization method to be used
    "coef": coef,  # Regularization coefficient to be used
    "solver": "LSMR_ini",  # Solver for the inversion
    "proj": 'EPSG:4326',  # EPSG system of the given coordinates
    "interp": "nearest",  # Interpolation method used to load the pixel when it is not in the dataset
    "visual": show | save,  # If the observations data need to be returned
}

start = time.time()
# Load pixel data
data, mean, dates_range = cube.load_pixel(lon, lat, rolling_mean=obs_filt, **load_pixel_kwargs)

# Prepare interpolation dates
first_date_interpol, last_date_interpol = cube.prepare_interpolation_date()
interpolation_kwargs.update({"first_date_interpol": first_date_interpol, "last_date_interpol": last_date_interpol})

stop = time.time()
print(f"Prepping Interpolation took {round((stop - start), 4)} s")
